In [ ]:
import scipy
from mortality.simulator import simulator

In [ ]:
pr = 'mortality'
with open(pr+'/prior.pkl', 'rb') as handle:
    prior = pickle.load(handle)
prior, theta_numel, prior_returns_numpy = utils.user_input_checks.process_prior(prior)
simulator = utils.user_input_checks.process_simulator(simulator, prior, is_numpy_simulator=True)
x_o = np.load(pr+'/observations/observations.npy')
x_o = torch.Tensor(x_o)
with open(pr+"/posterior.pkl", "rb") as handle:
    posterior = pickle.load(handle)
posterior_samples = posterior.sample(sample_shape=(1000,))
x_pp = simulator(posterior_samples)
x_pp = x_pp[~torch.any(x_pp.isnan(),dim=1)].numpy()
x_pp_mean = np.mean(x_pp, axis=0)

In [ ]:
samples_per_census = 1000
all_y = []
for census_i, census_t in enumerate(census_yrs):
    mean = x_pp_mean[census_i]
    var = x_pp_mean[census_i + 5]
    skew = x_pp_mean[census_i + 10]
    dist_samples = scipy.stats.skewnorm.rvs(skew, loc=mean, scale=np.sqrt(var), size=samples_per_census)
    all_y.append(np.where(dist_samples<0, 0, dist_samples))

In [ ]:
fig, ax = plt.subplots(1, figsize=(9,6))
ax_fontsize = 12
    
ax.scatter(mortality_o[0].to_numpy(), mortality_o[1].to_numpy(), color='k')

parts = ax.violinplot(all_y, positions=census_yrs, showextrema=False);
for pc in parts['bodies']:
    pc.set_facecolor('limegreen')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)

ax.set_ylim(-0.05, 1)
ax.set_xlim(1,15)
ax.set_ylabel("mortality rate", fontsize=ax_fontsize)
ax.set_xlabel("age", fontsize=ax_fontsize)
fn = f"sbi_figs/mortality_viz.png"
fig.savefig(fn, bbox_inches='tight')